In [1]:
from __future__ import annotations

from enum import Enum
from collections.abc import Sequence

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import CircuitInstruction
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit.library.standard_gates import IGate
from qiskit.providers import Backend
from qiskit.providers.fake_provider import FakeHanoiV2 as FakeHanoi

from circuit_knitting.cutting.qpd import decompose_qpd_instructions
from circuit_knitting.cutting import cut_gates

### Create a function for scoring gates based on how much depth would be reduced by removing them

In [2]:
def evaluate_cuts(
    circuit: QuantumCircuit,
    backend: Backend,
    gate_ids: Sequence[int] | None = None,
    initial_layout: Sequence[int] | None = None,
) -> list[tuple[int, int]]:
    circuit = circuit.copy()
    supported_gates = {"rxx", "ryy", "rzz", "crx", "cry", "crz", "cx", "cz"}

    input_depth = transpile(
        circuit, backend=backend, initial_layout=initial_layout
    ).depth()

    if gate_ids is None:
        gate_ids = list(range(len(circuit.data)))

    cut_scores = []
    for idx in gate_ids:
        if circuit[idx].operation.name not in supported_gates:
            continue
        inst_to_check = circuit[idx]
        del circuit.data[idx]
        cut_score = (
            input_depth
            - transpile(circuit, backend=backend, initial_layout=initial_layout).depth()
        )
        cut_scores.append((idx, cut_score))
        circuit.data.insert(idx, inst_to_check)

    return sorted(cut_scores, key=lambda x: x[1], reverse=True)

### Create a circuit and note the depth

In [7]:
backend = FakeHanoi()
circuit = EfficientSU2(16, entanglement="sca", reps=6).decompose()
original_circuit_depth = transpile(circuit, backend=backend).depth()
print(f"Original circuit depth after transpile: {original_circuit_depth}")

Original circuit depth after transpile: 298


### Get the top N cuts in one sweep, remove them, and observe the depth reduction

In [8]:
NUM_CUTS = 5

cut_scores = evaluate_cuts(circuit, backend)

cut_indices = [cut_scores[i][0] for i in range(NUM_CUTS)]
qpd_circuit, bases = cut_gates(circuit, cut_indices)

for idx in cut_indices:
    qpd_circuit[idx].operation.basis_id = 0
    inst = qpd_circuit[idx].operation

qpd_circuit_dx = decompose_qpd_instructions(qpd_circuit, [[idx] for idx in cut_indices])

# Transpile the decomposed circuit to the same layout
transpiled_qpd_circuit = transpile(qpd_circuit_dx, backend=backend)

print(f"QPD subexperiment depth after transpile: {transpiled_qpd_circuit.depth()}")

QPD subexperiment depth after transpile: 190


### Get the top N cuts in N sweeps, choose the best gate to cut at each sweep, and observe depth reduction

In [9]:
tmp_circ = circuit.copy()
cut_indices = []
for num_cuts in range(NUM_CUTS):
    cut_scores = evaluate_cuts(tmp_circ, backend)
    best_idx = cut_scores[0][0]
    cut_indices.append(best_idx)
    qubit0 = tmp_circ.find_bit(tmp_circ.data[best_idx].qubits[0]).index
    tmp_circ.data[best_idx] = CircuitInstruction(IGate(), qubits=(qubit0,))

### By doing an iterative, greedy approach, we can find a decent cutting scheme

In [10]:
qpd_circuit, bases = cut_gates(circuit, cut_indices)

for idx in cut_indices:
    qpd_circuit[idx].operation.basis_id = 0
    inst = qpd_circuit[idx].operation

qpd_circuit_dx = decompose_qpd_instructions(qpd_circuit, [[idx] for idx in cut_indices])

# Transpile the decomposed circuit to the same layout
transpiled_qpd_circuit = transpile(qpd_circuit_dx, backend=backend)

print(f"QPD subexperiment depth after transpile: {transpiled_qpd_circuit.depth()}")

QPD subexperiment depth after transpile: 100


In [ ]:
class MyClass:
    